In [2]:
# 구글 드라이브 연결

from google.colab import drive
import os

drive.mount('/content/drive')

project_path = '/content/drive/MyDrive/examples/2장/data'
os.makedirs(project_path, exist_ok=True)

# 현재 경로를 /content/drive로 설정
os.chdir(project_path)
print("현재 작업 디렉토리:", os.getcwd())
os.listdir(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
현재 작업 디렉토리: /content/drive/MyDrive/examples/2장/data


['shipsnet.json',
 'dance.mov',
 'heart.png',
 'sudoku.jpeg',
 'dog.png',
 'cat.png',
 'vtest.avi',
 'titanic.csv']

In [ ]:
# prompt: heart.png 파일을 바이트로 읽어와 출력 (헤더포함)'

!cat heart.png | xxd -b


00000000: 10001001 01010000 01001110 01000111 00001101 00001010  .PNG..
00000006: 00011010 00001010 00000000 00000000 00000000 00001101  ......
0000000c: 01001001 01001000 01000100 01010010 00000000 00000000  IHDR..
00000012: 00000000 00001010 00000000 00000000 00000000 00001010  ......
00000018: 00001000 00000110 00000000 00000000 00000000 10001101  ......
0000001e: 00110010 11001111 10111101 00000000 00000000 00000000  2.....
00000024: 00111001 01110100 01000101 01011000 01110100 01010011  9tEXtS
0000002a: 01101111 01100110 01110100 01110111 01100001 01110010  oftwar
00000030: 01100101 00000000 01001101 01100001 01110100 01110000  e.Matp
00000036: 01101100 01101111 01110100 01101100 01101001 01100010  lotlib
0000003c: 00100000 01110110 01100101 01110010 01110011 01101001   versi
00000042: 01101111 01101110 00110011 00101110 00110111 00101110  on3.7.
00000048: 00110001 00101100 00100000 01101000 01110100 01110100  1, htt
0000004e: 01110000 01110011 00111010 00101111 00101111 01101101 

In [ ]:
def xxd_format(binary_data):
    # 주소, 16진수, 그리고 ASCII 형태로 데이터를 출력하는 함수
    result = []
    for i in range(0, len(binary_data), 16):
        chunk = binary_data[i:i + 16]
        hex_chunk = ' '.join(f'{b:02x}' for b in chunk)
        ascii_chunk = ''.join(chr(b) if 32 <= b < 127 else '.' for b in chunk)
        result.append(f'{i:08x}: {hex_chunk:<48} {ascii_chunk}')
    return '\n'.join(result)

# 이미지 파일을 바이너리 모드로 읽기
with open('heart.png', 'rb') as file:
    binary_data = file.read()

# xxd 형식으로 출력
formatted_output = xxd_format(binary_data)
print(formatted_output)

00000000: 89 50 4e 47 0d 0a 1a 0a 00 00 00 0d 49 48 44 52  .PNG........IHDR
00000010: 00 00 00 0a 00 00 00 0a 08 06 00 00 00 8d 32 cf  ..............2.
00000020: bd 00 00 00 39 74 45 58 74 53 6f 66 74 77 61 72  ....9tEXtSoftwar
00000030: 65 00 4d 61 74 70 6c 6f 74 6c 69 62 20 76 65 72  e.Matplotlib ver
00000040: 73 69 6f 6e 33 2e 37 2e 31 2c 20 68 74 74 70 73  sion3.7.1, https
00000050: 3a 2f 2f 6d 61 74 70 6c 6f 74 6c 69 62 2e 6f 72  ://matplotlib.or
00000060: 67 2f db 0a 08 87 00 00 00 09 70 48 59 73 00 00  g/........pHYs..
00000070: 0f 61 00 00 0f 61 01 a8 3f a7 69 00 00 00 40 49  .a...a..?.i...@I
00000080: 44 41 54 78 9c 63 64 60 60 f8 cf 40 04 60 22 46  DATx.cd``..@.`"F
00000090: 11 8a c2 ff ff 31 0d 46 16 63 42 16 40 96 40 17  .....1.F.cB.@.@.
000000a0: 63 c1 67 0a 86 d5 8c 8c 8c 38 dd 06 93 63 42 17  c.g......8...cB.
000000b0: c0 a6 08 45 21 ba 04 ba 46 46 06 6a 87 23 00 52  ...E!...FF.j.#.R
000000c0: 74 17 10 a6 f8 4c 13 00 00 00 00 49 45 4e 44 ae  t....L.....IEND.
000000d0: 42

In [ ]:
# 이미지 파일을 바이너리 모드로 읽기
with open('heart.png', 'rb') as file:
    binary_data = file.read()

# IDAT 청크 추출: IDAT 데이터는 일반적으로 zlib 압축 알고리즘을 사용하여 압축되어 있음
def extract_idat_chunks(binary_data):
    idat_data = bytearray()
    offset = 8  # PNG 시그니처 이후
    while offset < len(binary_data):
        length = int.from_bytes(binary_data[offset:offset+4], byteorder='big')
        chunk_type = binary_data[offset+4:offset+8]
        chunk_data = binary_data[offset+8:offset+8+length]
        chunk_crc = binary_data[offset+8+length:offset+12+length]

        if chunk_type == b'IDAT':
            idat_data.extend(chunk_data)

        offset += 12 + length
        if chunk_type == b'IEND':
            break
    return idat_data

idat_data = extract_idat_chunks(binary_data)

# xxd 형식으로 출력하는 함수
def xxd_format(binary_data):
    result = []
    for i in range(0, len(binary_data), 16):
        chunk = binary_data[i:i + 16]
        hex_chunk = ' '.join(f'{b:02x}' for b in chunk)
        ascii_chunk = ''.join(chr(b) if 32 <= b < 127 else '.' for b in chunk)
        result.append(f'{i:08x}: {hex_chunk:<48} {ascii_chunk}')
    return '\n'.join(result)

# IDAT 데이터 16진수 형식으로 출력
formatted_output = xxd_format(idat_data)
print(formatted_output)

00000000: 78 9c 63 64 60 60 f8 cf 40 04 60 22 46 11 8a c2  x.cd``..@.`"F...
00000010: ff ff 31 0d 46 16 63 42 16 40 96 40 17 63 c1 67  ..1.F.cB.@.@.c.g
00000020: 0a 86 d5 8c 8c 8c 38 dd 06 93 63 42 17 c0 a6 08  ......8...cB....
00000030: 45 21 ba 04 ba 46 46 06 6a 87 23 00 52 74 17 10  E!...FF.j.#.Rt..
